<a href="https://colab.research.google.com/github/Shrey327/Machine_Learning/blob/main/basic_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

In [2]:
!pip install -U langchain-google-genai

In [21]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash-preview-04-17", model_provider="google_genai")

Enter API key for Google Gemini: ··········


In [4]:
!pip install -qU langchain-huggingface

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
!pip install -U langchain-community pypdf

In [7]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader(
    path = "/content",
    glob = "**/[!.]*.pdf",
    silent_errors = True,
    load_hidden = False,
    recursive = False,
    extract_images = False,
    password = None,
    mode = "page",
    headers = None,
    extraction_mode = "plain",
)

In [8]:
docs = loader.load()

In [9]:
!pip install --quiet langchain_experimental langchain_openai

In [10]:
docs_content = [doc.page_content for doc in docs]

In [11]:
len(docs_content)

1

#**Gradient**

In this method, the gradient of distance is used to split chunks along with the percentile method. This method is useful when chunks are highly correlated with each other or specific to a domain e.g. legal or medical. The idea is to apply anomaly detection on gradient array so that the distribution become wider and easy to identify boundaries in highly semantic data. Similar to the percentile method, the split can be adjusted by the keyword argument breakpoint_threshold_amount which expects a number between 0.0 and 100.0, the default value is 95.0.

source: Langchain

The docs_content variable, which contains the text extracted from the PDF, is being passed to the text_splitter.create_documents method, which in turn calls the embedding model. The presence of the invalid character '\ud835' within the text causes errors


In [12]:
from langchain_experimental.text_splitter import SemanticChunker

text_splitter = SemanticChunker(
    embeddings, breakpoint_threshold_type="gradient"
)

In [13]:
cleaned_docs_content = []
for content in docs_content:
    cleaned_content = content.encode('utf-8', 'ignore').decode('utf-8')
    cleaned_docs_content.append(cleaned_content)

docs = text_splitter.create_documents(cleaned_docs_content)
print(docs[0].page_content)

Shreyansh Pathak
/githubgithub.com/Shrey327 /linkedinlinkedin.com/in/Shreyansh Pathak /envel⌢peshreyansh.pathak27@gmail.com
Education
Dayananda Sagar College Of Engineering Jun. 2027
B.E AIML Current GPA: 8.67
Satyam INTL School (C.B.S.E) Jun. 2022
87.5%
Coursework
Courses: Object-Oriented Programming, Data Structures & Algorithms, Embedded Systems, Discrete Math,
Linear Algebra, Calculus, Physics, Probability & Statistics, Operating Systems, Database Management
Systems, Computer Networks, Machine Learning, Deep Learning, Natural Language Processing, Web
Development
Skills
Languages: C/C++, Python, Java, JavaScript/TypeScript, HTML/CSS
Tools: Git/GitHub, Google Colab, VS Code
Frameworks: React, Material-UI, FastAPI, TailwindCSS
Libraries: pandas, NumPy, Matplotlib, TensorFlow, PyTorch
Projects
3RVision | Python, TensorFlow, Flask, Google Colab, Git Apr. 2025
• Built and trained a ResNet50-based computer vision model to classify product materials for sustainable lifecycle
analysis. • Im

In [14]:
pip install -qU langchain-qdrant

In [15]:
!pip install -qU langchain-community

In [16]:
!pip install faiss-cpu

In [17]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [18]:
document_ids = vector_store.add_documents(documents=docs)

In [53]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from IPython.display import display, Markdown

retriever = vector_store.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
prompt = hub.pull("rlm/rag-prompt")

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


results=rag_chain.invoke("briefly explain in why Shreyansh should be hired for machine learning internship")
display(Markdown(results))

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Based on the context, Shreyansh should be hired due to his strong academic background in AIML with relevant coursework and a high GPA. He possesses practical skills in key ML tools like Python, TensorFlow, and PyTorch, demonstrated through significant projects in computer vision, NLP, and classification. These projects showcase his ability to apply machine learning techniques hands-on.

In [24]:
!pip install mem0ai

In [25]:
from typing import List, Dict
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from mem0 import MemoryClient
from langchain_core.output_parsers import StrOutputParser

In [26]:
if not os.environ.get("MEM0_API_KEY"):
  os.environ["MEM0_API_KEY"] = getpass.getpass("Enter API key for mem0: ")

Enter API key for mem0: ··········


In [27]:
mem0 = MemoryClient()

In [55]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="""tou are a helpful assistant with access to a knowledge base. Your task is to answer user questions accurately based on the retrieved context provided to you.
Instructions:

Always respond based on the retrieved context when it's relevant to the query.
If the context doesn't contain the information needed to answer the question, clearly state that you don't have that specific information.
Do not make up or infer information that isn't supported by the context.
Keep responses concise and focused on addressing the user's question.
When quoting directly from the context, use quotation marks.
Always cite the specific document/source when providing information from the context.

Retrieved Context:
{{CONTEXT}}
User Query:
{{QUERY}}
Response:"""+results),
    MessagesPlaceholder(variable_name="context"),
    HumanMessage(content="{input}")
])

In [56]:
def chat_turn(user_input: str, user_id: str) -> str:
    # Retrieve context
    context = retrieve_context(user_input, user_id)

    # Generate response
    response = generate_response(user_input, context)

    # Save interaction
    save_interaction(user_id, user_input, response)

    return response

In [57]:
def retrieve_context(query: str, user_id: str) -> List[Dict]:
    """Retrieve relevant context from Mem0"""
    memories = mem0.search(query, user_id=user_id)
    seralized_memories = ' '.join([mem["memory"] for mem in memories])
    context = [
        {
            "role": "system",
            "content": f"Relevant information: {seralized_memories}"
        },
        {
            "role": "user",
            "content": query
        }
    ]
    return context

def generate_response(input: str, context: List[Dict]) -> str:
    """Generate a response using the language model"""
    chain = prompt | llm
    response = chain.invoke({
        "context": context,
        "input": input
    })
    return response.content

def save_interaction(user_id: str, user_input: str, assistant_response: str):
    """Save the interaction to Mem0"""
    interaction = [
        {
          "role": "user",
          "content": user_input
        },
        {
            "role": "assistant",
            "content": assistant_response
        }
    ]
    mem0.add(interaction, user_id=user_id)

In [59]:
if __name__ == "__main__":

    user_id = "Shreyansh"

    while True:
        user_input = input("You: ")
        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("Agent: Thank you")
            break

        response = chat_turn(user_input, user_id)
        print(f"Agent: {response}")

You: tell me about shreyansh


/usr/local/lib/python3.11/dist-packages/mem0/client/main.py:34: DeprecationWarning: output_format='v1.0' is deprecated therefore setting it to 'v1.1' by default.Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)


Agent: Based on the provided context, Shreyansh Pathak is a student pursuing a B.E. in AIML at Dayananda Sagar College Of Engineering. He has a strong academic background with relevant coursework and a high GPA in AIML. Shreyansh possesses practical skills in key ML tools like Python, TensorFlow, and PyTorch, and is proficient with various tools and frameworks. He also has skills in Java and JavaScript. He has demonstrated his skills through significant projects in computer vision, natural language processing, classification, and data analysis. Additionally, Shreyansh has experience as a team leader in a technical club.
You: tell me about his skills in ml


/usr/local/lib/python3.11/dist-packages/mem0/client/main.py:34: DeprecationWarning: output_format='v1.0' is deprecated therefore setting it to 'v1.1' by default.Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)


Agent: Based on the context, Shreyansh's skills in ML include:

*   A strong academic background in AIML (Artificial Intelligence and Machine Learning).
*   Practical skills in key ML tools such as Python, TensorFlow, and PyTorch.
*   Proficiency with various tools and frameworks (relevant to ML).
*   Demonstrated ability to apply machine learning techniques hands-on through significant projects in areas like computer vision, natural language processing (NLP), and classification/data analysis.
You: exit
Agent: Thank you
